In [1]:
import pandas as pd
from clickhouse_driver import Client
from datetime import datetime
import logging

def read_clickhouse_to_dataframe(host,port,user,password,database):
    query = 'SELECT clientid,sendid,subscriberkey,emailaddress,subscriberid,listid,eventdate,eventtype,batchid,filedate,source_name,file_name,insertion_date FROM send'
    # Connect to ClickHouse server
    client = Client(host=host, port=port, user=user, password=password, database=database)
    try:
        # Execute SQL query to fetch data
        result = client.execute(query)
        columns = ['clientid','sendid','subscriberkey','emailaddress','subscriberid','listid','eventdate','eventtype','batchid','filedate','source_name','file_name','insertion_date']
        df = pd.DataFrame(result, columns=columns)
        df=df.to_dict(orient="records")

    finally:
        # Close ClickHouse connection
        client.disconnect()

    return df


def main_new(dfs,host,port,user,password,database):
    df = read_clickhouse_to_dataframe(host,port,user,password,database)
    # df=df[10:15]
    start_percentage = 10
    end_percentage = 15
    
    total_rows = len(df)
    start_index = int(total_rows * (start_percentage / 100))
    end_index = int(total_rows * (end_percentage / 100))
    df = df[start_index:end_index]
    lis=[]
    for i in range(len(df)):
        try:
            dfs[i]['clientid']=df[i]['clientid']
            dfs[i]['sendid']=df[i]['sendid']
            dfs[i]['subscriberkey']=df[i]['subscriberkey']
            dfs[i]['emailaddress']=df[i]['emailaddress']
            dfs[i]['subscriberid']=df[i]['subscriberid']
            dfs[i]['listid']=df[i]['listid']
            dfs[i]['eventdate']=df[i]['eventdate']
            dfs[i]['batchid']=df[i]['batchid']
            dfs[i]['filedate']=df[i]['filedate']
            dfs[i]['file_name']=df[i]['file_name']
            dfs[i]['source_name']=df[i]['source_name']
            dfs[i]['insertion_date']=str(datetime.now().date())
            lis.append(dfs[i])
        except Exception as e:
            logging.info(e)
    return lis